<h1> Create the FB problem table </h1>

In [1]:
from __future__ import division
import json
import pandas as pd
import re, collections
import numpy as np
import os
base_dir=os.path.expanduser('~')
import datetime
import time
import psycopg2

In [11]:
#connect
conn = psycopg2.connect("host=localhost port=5432 dbname=fb")
cur = conn.cursor()

In [12]:
#Create Friend table
cur.execute(\
"CREATE TABLE Friend (id serial PRIMARY KEY,\
    u_id integer,\
    f_id integer\
    );")
conn.commit()

In [13]:
#Create Friend table
cur.execute(\
"CREATE TABLE Liked (id serial PRIMARY KEY,\
    u_id integer,\
    page_id integer\
    );")
conn.commit()

<h1> Add entries for table </h1>

In [14]:
users=[1,1,2,2,1,3,3,2,1]
friends=[2,3,3,4,4,2,6,1,6]
ul=[1,1,2,2,1,1,3,3,4,4,4,2,1]#13
liked=[4,5,5,6,7,1,1,8,9,0,3,2,7]

In [ ]:
for u,f in zip(users,friends):
    cur.execute("INSERT INTO Friend (u_id,f_id) VALUES (%s,%s);",(u,f))
conn.commit()

In [17]:
for u,p in zip(ul,liked):
    cur.execute("INSERT INTO Liked (u_id,page_id) VALUES (%s,%s);",(u,p))
conn.commit()

<h1> Solution </h1>

create a table with pages that "I Liked" and join on a table that your "Friends Liked" ON u_id=u_id and I_liked=Friend_like, return the rows that your friends like but what you liked is null

select FL.u_id, array_agg(FL.friend_liked) as rec from (select friend.u_id,liked.page_id as friend_liked from Liked join Friend on Liked.u_id=Friend.f_id) as FL left join (select u_id,page_id as i_liked from Liked) as IL on FL.u_id=IL.u_id AND FL.friend_liked=IL.i_liked WHERE IL.i_liked is NULL group by FL.u_id;

<p>
1. (select Friend.u_id,liked.page_id as friend_liked from Liked join Friend on Liked.u_id=Friend.f_id) as FL
    <br> This gets the pages that the friend of each user like  <br> 
2.  (select u_id,page_id as i_liked from Liked)
    <br> Selects the pages I liked  <br> 
3. Left join on u_id and i_liked=friend_liked liked where i_liked is null
</p>